In [1]:
import os
import urllib.request
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS

os.environ['OPENAI_API_KEY'] = "여기에 API 키를 넣어주세요"

In [2]:
urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch06/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf", filename="2023_북한인권보고서.pdf")

('2023_북한인권보고서.pdf', <http.client.HTTPMessage at 0x11a315fc0>)

In [3]:
loader = PyPDFLoader('2023_북한인권보고서.pdf')
pages = loader.load_and_split()
print('청크의 수:', len(pages))

청크의 수: 445


In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

In [6]:
splitted_docs = text_splitter.split_documents(pages)
print('분할된 청크의 수:', len(splitted_docs))

분할된 청크의 수: 502


In [7]:
chunks = [splitted_doc.page_content for splitted_doc in splitted_docs]
print('청크의 최대 길이 :',max(len(chunk) for chunk in chunks))
print('청크의 최소 길이 :',min(len(chunk) for chunk in chunks))
print('청크의 평균 길이 :',sum(map(len, chunks))/len(chunks))

청크의 최대 길이 : 1000
청크의 최소 길이 : 6
청크의 평균 길이 : 745.3844621513945


In [8]:
db = Chroma.from_documents(splitted_docs, OpenAIEmbeddings())
print('문서의 수:', db._collection.count())

/var/folders/60/_7v7nlqn6z3bx_g52j1b3vhw0000gn/T/ipykernel_50921/1222124650.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  db = Chroma.from_documents(splitted_docs, OpenAIEmbeddings())


문서의 수: 502


In [9]:
question = '북한의 교육과정'
docs = db.similarity_search(question)
print('문서의 수:', len(docs))

문서의 수: 4


In [10]:
for doc in docs:
    print(doc)
    print('--' * 10)

page_content='를	받아	들이는	것으로	보인다. 	
309		북한의	학제는	2012년	전반적	의무교육(유치원 	1년,	소학교	5년,	초급중학교 	3년,	고급중학교 	3년)으로 	
개편되었는데,	 학제개편 	이전에는 	초급중학교와 	고급중학교를 	통합하여 	중학교	6년	과정(1972년~2011
년)으로	운영하였고,	 중학교	또는	고등중학교라고 	칭하였다.(통일부 	국립통일교육원,	 『북한의	이해』,	
2022)' metadata={'page': 283, 'source': '2023_북한인권보고서.pdf'}
--------------------
page_content='2023 북한인권보고서40명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과
서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교
과서가 모든 학생에게 충분히 제공되지 않고 학년을 마치면 다음 학
년에 교과서를 물려주어야 했다는 사례가 다수 수집되었다. 소학교
부터 학교운영비, 꼬마계획 등의 비용을 내야했다는 진술이 꾸준히 
수집되고 있는데, 학교시설 현대화 작업이 진행되면서 학교꾸리기 
비용이 증가했다고 한다. 학교에서 요구하는 돈이나 물품은 교원에 
의해 사실상 강제되고 있었는데, 비용을 내지 못하는 경우 동급생들 
앞에서 망신을 주거나 비판하여 형편이 어려운 학생들은 학교를 그
만두는 선택을 하는 경우가 많다고 한다. 또한 도시와 농촌 간 교육
환경의 차이가 크며 대학입학에서 출신성분에 의한 차별이 있고, 교
육기회의 제공에도 경제력이 영향을 미치고 있어 성분·지역·경제
력에 따른 차별이 존재하는 것으로 나타났다. 교육환경도 열악한데, 
학교시설의 현대화 작업에도 불구하고 양호실, 도서관, 위생시설이 
없는 학교도 많은 것으로 보인다. 교원에 대한 경제적 보상도 적절
히 이루어지지 않아, 교원들은 생계를 유지하기 위해 잘사는 학부모
의 원조를 받거나 자신의 텃밭에 학생을 동원시키고 있어 학생들은 
제대로 된 교육여건을 보장받지 못하고 있는 것으로 나타났다. 

In [11]:
db_to_file = Chroma.from_documents(splitted_docs, OpenAIEmbeddings(),
                                   persist_directory = './chroma_test.db')
print('문서의 수:', db_to_file._collection.count())

문서의 수: 502


In [12]:
db_from_file = Chroma(persist_directory='./chroma_test.db',
                      embedding_function=OpenAIEmbeddings())
print('문서의 수:', db_from_file._collection.count())

문서의 수: 502


/var/folders/60/_7v7nlqn6z3bx_g52j1b3vhw0000gn/T/ipykernel_50921/2367151211.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  db_from_file = Chroma(persist_directory='./chroma_test.db',


In [13]:
question = '북한의 교육 과정'
top_three_docs = db_from_file.similarity_search_with_relevance_scores(question, 
                                                                      k=3)
for doc in top_three_docs:
    print(doc)
    print('--' * 10)

(Document(metadata={'page': 283, 'source': '2023_북한인권보고서.pdf'}, page_content='를\t받아\t들이는\t것으로\t보인다. \t\n309\t\t북한의\t학제는\t2012년\t전반적\t의무교육(유치원 \t1년,\t소학교\t5년,\t초급중학교 \t3년,\t고급중학교 \t3년)으로 \t\n개편되었는데,\t 학제개편 \t이전에는 \t초급중학교와 \t고급중학교를 \t통합하여 \t중학교\t6년\t과정(1972년~2011\n년)으로\t운영하였고,\t 중학교\t또는\t고등중학교라고 \t칭하였다.(통일부 \t국립통일교육원,\t 『북한의\t이해』,\t\n2022)'), 0.8302666120719884)
--------------------
(Document(metadata={'page': 41, 'source': '2023_북한인권보고서.pdf'}, page_content='2023 북한인권보고서40명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과\n서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교\n과서가 모든 학생에게 충분히 제공되지 않고 학년을 마치면 다음 학\n년에 교과서를 물려주어야 했다는 사례가 다수 수집되었다. 소학교\n부터 학교운영비, 꼬마계획 등의 비용을 내야했다는 진술이 꾸준히 \n수집되고 있는데, 학교시설 현대화 작업이 진행되면서 학교꾸리기 \n비용이 증가했다고 한다. 학교에서 요구하는 돈이나 물품은 교원에 \n의해 사실상 강제되고 있었는데, 비용을 내지 못하는 경우 동급생들 \n앞에서 망신을 주거나 비판하여 형편이 어려운 학생들은 학교를 그\n만두는 선택을 하는 경우가 많다고 한다. 또한 도시와 농촌 간 교육\n환경의 차이가 크며 대학입학에서 출신성분에 의한 차별이 있고, 교\n육기회의 제공에도 경제력이 영향을 미치고 있어 성분·지역·경제\n력에 따른 차별이 존재하는 것으로 나타났다. 교육환경도 열악한데, \n학교시설의 현대화 작업에도 불구하고 양호실, 도서관, 위생시

In [14]:
faiss_db = FAISS.from_documents(splitted_docs, OpenAIEmbeddings())
print('문서의 수:', faiss_db.index.ntotal)

문서의 수: 502


In [16]:
faiss_db.save_local('faiss_index')

new_db_faiss = FAISS.load_local('faiss_index',
                                OpenAIEmbeddings(),
                                allow_dangerous_deserialization=True)

In [17]:
question = '북한의 교육 과정'
docs = new_db_faiss.similarity_search(question)

for doc in docs:
    print(doc)
    print('--' * 10)

page_content='를	받아	들이는	것으로	보인다. 	
309		북한의	학제는	2012년	전반적	의무교육(유치원 	1년,	소학교	5년,	초급중학교 	3년,	고급중학교 	3년)으로 	
개편되었는데,	 학제개편 	이전에는 	초급중학교와 	고급중학교를 	통합하여 	중학교	6년	과정(1972년~2011
년)으로	운영하였고,	 중학교	또는	고등중학교라고 	칭하였다.(통일부 	국립통일교육원,	 『북한의	이해』,	
2022)' metadata={'source': '2023_북한인권보고서.pdf', 'page': 283}
--------------------
page_content='2023 북한인권보고서40명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과
서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교
과서가 모든 학생에게 충분히 제공되지 않고 학년을 마치면 다음 학
년에 교과서를 물려주어야 했다는 사례가 다수 수집되었다. 소학교
부터 학교운영비, 꼬마계획 등의 비용을 내야했다는 진술이 꾸준히 
수집되고 있는데, 학교시설 현대화 작업이 진행되면서 학교꾸리기 
비용이 증가했다고 한다. 학교에서 요구하는 돈이나 물품은 교원에 
의해 사실상 강제되고 있었는데, 비용을 내지 못하는 경우 동급생들 
앞에서 망신을 주거나 비판하여 형편이 어려운 학생들은 학교를 그
만두는 선택을 하는 경우가 많다고 한다. 또한 도시와 농촌 간 교육
환경의 차이가 크며 대학입학에서 출신성분에 의한 차별이 있고, 교
육기회의 제공에도 경제력이 영향을 미치고 있어 성분·지역·경제
력에 따른 차별이 존재하는 것으로 나타났다. 교육환경도 열악한데, 
학교시설의 현대화 작업에도 불구하고 양호실, 도서관, 위생시설이 
없는 학교도 많은 것으로 보인다. 교원에 대한 경제적 보상도 적절
히 이루어지지 않아, 교원들은 생계를 유지하기 위해 잘사는 학부모
의 원조를 받거나 자신의 텃밭에 학생을 동원시키고 있어 학생들은 
제대로 된 교육여건을 보장받지 못하고 있는 것으로 나타났다. 